In [1]:
import pandas as pd

In [2]:
Players = pd.read_csv('Data/Players.csv')
Players

,Unnamed: 0,Player,height,weight,collage,born,birth_city,birth_state
0,0,Curly Armstrong,180.0,77.0,Indiana University,1918.0,NaN,NaN
1,1,Cliff Barker,188.0,83.0,University of Kentucky,1921.0,Yorktown,Indiana
2,2,Leo Barnhorst,193.0,86.0,University of Notre Dame,1924.0,NaN,NaN
3,3,Ed Bartels,196.0,88.0,North Carolina State University,1925.0,NaN,NaN
4,4,Ralph Beard,178.0,79.0,University of Kentucky,1927.0,Hardinsburg,Kentucky
...,...,...,...,...,...,...,...,...
3917,3917,Troy Williams,198.0,97.0,South Carolina State University,1969.0,Columbia,South Carolina
3918,3918,Kyle Wiltjer,208.0,108.0,Gonzaga University,1992.0,Portland,Oregon
3919,3919,Stephen Zimmerman,213.0,108.0,"University of Nevada, Las Vegas",1996.0,Hendersonville,Tennessee
3920,3920,Paul Zipser,203.0,97.0,NaN,1994.0,Heidelberg,Germany


In [3]:
players = Players.drop(columns= ['Unnamed: 0'], axis=1)
players

,Player,height,weight,collage,born,birth_city,birth_state
0,Curly Armstrong,180.0,77.0,Indiana University,1918.0,NaN,NaN
1,Cliff Barker,188.0,83.0,University of Kentucky,1921.0,Yorktown,Indiana
2,Leo Barnhorst,193.0,86.0,University of Notre Dame,1924.0,NaN,NaN
3,Ed Bartels,196.0,88.0,North Carolina State University,1925.0,NaN,NaN
4,Ralph Beard,178.0,79.0,University of Kentucky,1927.0,Hardinsburg,Kentucky
...,...,...,...,...,...,...,...
3917,Troy Williams,198.0,97.0,South Carolina State University,1969.0,Columbia,South Carolina
3918,Kyle Wiltjer,208.0,108.0,Gonzaga University,1992.0,Portland,Oregon
3919,Stephen Zimmerman,213.0,108.0,"University of Nevada, Las Vegas",1996.0,Hendersonville,Tennessee
3920,Paul Zipser,203.0,97.0,NaN,1994.0,Heidelberg,Germany


In [4]:
stats = pd.read_csv('Data/Seasons_stats.csv')
stats

,Unnamed: 0,Year,Player,Pos,Age,Tm,G,GS,MP,PER,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,0,1950.0,Curly Armstrong,G-F,31.0,FTW,63.0,NaN,NaN,NaN,...,0.705,NaN,NaN,NaN,176.0,NaN,NaN,NaN,217.0,458.0
1,1,1950.0,Cliff Barker,SG,29.0,INO,49.0,NaN,NaN,NaN,...,0.708,NaN,NaN,NaN,109.0,NaN,NaN,NaN,99.0,279.0
2,2,1950.0,Leo Barnhorst,SF,25.0,CHS,67.0,NaN,NaN,NaN,...,0.698,NaN,NaN,NaN,140.0,NaN,NaN,NaN,192.0,438.0
3,3,1950.0,Ed Bartels,F,24.0,TOT,15.0,NaN,NaN,NaN,...,0.559,NaN,NaN,NaN,20.0,NaN,NaN,NaN,29.0,63.0
4,4,1950.0,Ed Bartels,F,24.0,DNN,13.0,NaN,NaN,NaN,...,0.548,NaN,NaN,NaN,20.0,NaN,NaN,NaN,27.0,59.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24686,24686,2017.0,Cody Zeller,PF,24.0,CHO,62.0,58.0,1725.0,16.7,...,0.679,135.0,270.0,405.0,99.0,62.0,58.0,65.0,189.0,639.0
24687,24687,2017.0,Tyler Zeller,C,27.0,BOS,51.0,5.0,525.0,13.0,...,0.564,43.0,81.0,124.0,42.0,7.0,21.0,20.0,61.0,178.0
24688,24688,2017.0,Stephen Zimmerman,C,20.0,ORL,19.0,0.0,108.0,7.3,...,0.600,11.0,24.0,35.0,4.0,2.0,5.0,3.0,17.0,23.0
24689,24689,2017.0,Paul Zipser,SF,22.0,CHI,44.0,18.0,843.0,6.9,...,0.775,15.0,110.0,125.0,36.0,15.0,16.0,40.0,78.0,240.0


In [5]:
stats = stats.drop(columns= ['Unnamed: 0'], axis=1)
stats

,Year,Player,Pos,Age,Tm,G,GS,MP,PER,TS%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1950.0,Curly Armstrong,G-F,31.0,FTW,63.0,NaN,NaN,NaN,0.368,...,0.705,NaN,NaN,NaN,176.0,NaN,NaN,NaN,217.0,458.0
1,1950.0,Cliff Barker,SG,29.0,INO,49.0,NaN,NaN,NaN,0.435,...,0.708,NaN,NaN,NaN,109.0,NaN,NaN,NaN,99.0,279.0
2,1950.0,Leo Barnhorst,SF,25.0,CHS,67.0,NaN,NaN,NaN,0.394,...,0.698,NaN,NaN,NaN,140.0,NaN,NaN,NaN,192.0,438.0
3,1950.0,Ed Bartels,F,24.0,TOT,15.0,NaN,NaN,NaN,0.312,...,0.559,NaN,NaN,NaN,20.0,NaN,NaN,NaN,29.0,63.0
4,1950.0,Ed Bartels,F,24.0,DNN,13.0,NaN,NaN,NaN,0.308,...,0.548,NaN,NaN,NaN,20.0,NaN,NaN,NaN,27.0,59.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24686,2017.0,Cody Zeller,PF,24.0,CHO,62.0,58.0,1725.0,16.7,0.604,...,0.679,135.0,270.0,405.0,99.0,62.0,58.0,65.0,189.0,639.0
24687,2017.0,Tyler Zeller,C,27.0,BOS,51.0,5.0,525.0,13.0,0.508,...,0.564,43.0,81.0,124.0,42.0,7.0,21.0,20.0,61.0,178.0
24688,2017.0,Stephen Zimmerman,C,20.0,ORL,19.0,0.0,108.0,7.3,0.346,...,0.600,11.0,24.0,35.0,4.0,2.0,5.0,3.0,17.0,23.0
24689,2017.0,Paul Zipser,SF,22.0,CHI,44.0,18.0,843.0,6.9,0.503,...,0.775,15.0,110.0,125.0,36.0,15.0,16.0,40.0,78.0,240.0


In [6]:
len(stats.Player.unique())

3922

In [7]:
stats.isna().sum()

Year         67
Player       67
Pos          67
Age          75
Tm           67
G            67
GS         6458
MP          553
PER         590
TS%         153
3PAr       5852
FTr         166
ORB%       3899
DRB%       3899
TRB%       3120
AST%       2136
STL%       3899
BLK%       3899
TOV%       5109
USG%       5051
blanl     24691
OWS         106
DWS         106
WS          106
WS/48       590
blank2    24691
OBPM       3894
DBPM       3894
BPM        3894
VORP       3894
FG           67
FGA          67
FG%         166
3P         5764
3PA        5764
3P%        9275
2P           67
2PA          67
2P%         195
eFG%        166
FT           67
FTA          67
FT%         925
ORB        3894
DRB        3894
TRB         379
AST          67
STL        3894
BLK        3894
TOV        5046
PF           67
PTS          67
dtype: int64

In [8]:
stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24691 entries, 0 to 24690
Data columns (total 52 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Year    24624 non-null  float64
 1   Player  24624 non-null  object 
 2   Pos     24624 non-null  object 
 3   Age     24616 non-null  float64
 4   Tm      24624 non-null  object 
 5   G       24624 non-null  float64
 6   GS      18233 non-null  float64
 7   MP      24138 non-null  float64
 8   PER     24101 non-null  float64
 9   TS%     24538 non-null  float64
 10  3PAr    18839 non-null  float64
 11  FTr     24525 non-null  float64
 12  ORB%    20792 non-null  float64
 13  DRB%    20792 non-null  float64
 14  TRB%    21571 non-null  float64
 15  AST%    22555 non-null  float64
 16  STL%    20792 non-null  float64
 17  BLK%    20792 non-null  float64
 18  TOV%    19582 non-null  float64
 19  USG%    19640 non-null  float64
 20  blanl   0 non-null      float64
 21  OWS     24585 non-null  float64
 22

In [9]:
stats.loc[stats['Year'] == 1984]

,Year,Player,Pos,Age,Tm,G,GS,MP,PER,TS%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
7215,1984.0,Kareem Abdul-Jabbar*,C,36.0,LAL,80.0,80.0,2622.0,21.3,0.608,...,0.723,169.0,418.0,587.0,211.0,55.0,143.0,221.0,211.0,1717.0
7216,1984.0,Alvan Adams,C,29.0,PHO,70.0,13.0,1452.0,16.6,0.513,...,0.825,118.0,201.0,319.0,219.0,73.0,31.0,117.0,195.0,670.0
7217,1984.0,Mark Aguirre,SF,24.0,DAL,79.0,79.0,2900.0,23.5,0.572,...,0.749,161.0,308.0,469.0,358.0,80.0,22.0,285.0,246.0,2330.0
7218,1984.0,Danny Ainge,SG,24.0,BOS,71.0,3.0,1154.0,10.4,0.498,...,0.821,29.0,87.0,116.0,162.0,41.0,4.0,70.0,143.0,384.0
7219,1984.0,J.J. Anderson,SF,23.0,UTA,48.0,0.0,311.0,11.6,0.427,...,0.414,38.0,25.0,63.0,22.0,15.0,9.0,20.0,28.0,122.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7553,1984.0,Randy Wittman,SG,24.0,ATL,78.0,1.0,1071.0,9.3,0.517,...,0.609,14.0,57.0,71.0,71.0,17.0,0.0,32.0,82.0,350.0
7554,1984.0,Al Wood,SG,25.0,SEA,81.0,81.0,2236.0,14.9,0.545,...,0.823,94.0,181.0,275.0,166.0,64.0,32.0,126.0,207.0,1160.0
7555,1984.0,Mike Woodson,SG,25.0,KCK,71.0,12.0,1838.0,15.9,0.541,...,0.818,62.0,113.0,175.0,175.0,83.0,28.0,115.0,174.0,1027.0
7556,1984.0,Orlando Woolridge,SF,24.0,CHI,75.0,74.0,2544.0,15.9,0.567,...,0.715,130.0,239.0,369.0,136.0,71.0,60.0,188.0,253.0,1444.0


In [25]:
stats.loc[stats['Player'] == 'A.C. Green']

,Year,Player,Pos,Age,Tm,G,GS,MP,PER,TS%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
8035,1986.0,A.C. Green,PF,22.0,LAL,82.0,1.0,1542.0,11.8,0.564,...,0.611,160.0,221.0,381.0,54.0,49.0,49.0,99.0,229.0,521.0
8420,1987.0,A.C. Green,PF,23.0,LAL,79.0,72.0,2240.0,15.7,0.599,...,0.780,210.0,405.0,615.0,84.0,70.0,80.0,102.0,171.0,852.0
8807,1988.0,A.C. Green,PF,24.0,LAL,82.0,64.0,2636.0,14.5,0.581,...,0.773,245.0,465.0,710.0,93.0,87.0,45.0,120.0,204.0,937.0
9242,1989.0,A.C. Green,PF,25.0,LAL,82.0,82.0,2510.0,17.8,0.594,...,0.786,258.0,481.0,739.0,103.0,94.0,55.0,119.0,172.0,1088.0
9688,1990.0,A.C. Green,PF,26.0,LAL,82.0,82.0,2709.0,14.7,0.548,...,0.751,262.0,450.0,712.0,90.0,66.0,50.0,116.0,207.0,1061.0
10166,1991.0,A.C. Green,PF,27.0,LAL,82.0,21.0,2164.0,13.8,0.556,...,0.738,201.0,315.0,516.0,71.0,59.0,23.0,99.0,117.0,750.0
10617,1992.0,A.C. Green,PF,28.0,LAL,82.0,53.0,2902.0,16.7,0.556,...,0.744,306.0,456.0,762.0,117.0,91.0,36.0,111.0,141.0,1116.0
11060,1993.0,A.C. Green,PF,29.0,LAL,82.0,55.0,2819.0,16.3,0.603,...,0.739,287.0,424.0,711.0,116.0,88.0,39.0,116.0,149.0,1051.0
11529,1994.0,A.C. Green,PF,30.0,PHO,82.0,55.0,2825.0,17.0,0.555,...,0.735,275.0,478.0,753.0,137.0,70.0,38.0,100.0,142.0,1204.0
11999,1995.0,A.C. Green,SF,31.0,PHO,82.0,52.0,2687.0,14.2,0.596,...,0.732,194.0,475.0,669.0,127.0,55.0,31.0,114.0,146.0,916.0


In [28]:
stats.groupby('Player').mean()

,Year,Age,G,GS,MP,PER,TS%,3PAr,FTr,ORB%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
Player,,,,,,,,,,,,,,,,,,,,,
A.C. Green,1993.888889,29.888889,75.611111,50.277778,2169.111111,13.872222,0.543556,0.049000,0.437111,10.388889,...,0.708056,198.666667,364.055556,562.722222,81.611111,61.277778,31.222222,83.777778,143.388889,718.222222
A.J. Bramlett,2000.000000,23.000000,8.000000,0.000000,61.000000,-0.400000,0.190000,0.000000,0.000000,21.700000,...,NaN,12.000000,10.000000,22.000000,0.000000,1.000000,0.000000,3.000000,13.000000,8.000000
A.J. English,1991.500000,23.500000,75.500000,9.000000,1554.000000,11.550000,0.480000,0.047000,0.241000,4.900000,...,0.774000,70.000000,87.500000,157.500000,160.000000,28.500000,12.000000,101.500000,143.500000,751.000000
A.J. Guyton,2002.000000,23.000000,26.666667,4.666667,415.333333,4.366667,0.324000,0.371000,0.068333,1.366667,...,0.824000,7.333333,19.333333,26.666667,49.000000,6.666667,4.000000,20.666667,19.333333,147.333333
A.J. Hammons,2017.000000,24.000000,22.000000,0.000000,163.000000,8.400000,0.472000,0.238000,0.476000,5.400000,...,0.450000,8.000000,28.000000,36.000000,4.000000,1.000000,13.000000,10.000000,21.000000,48.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zelmo Beaty*,1967.125000,27.125000,71.250000,NaN,2293.500000,16.462500,0.525000,NaN,0.425875,7.400000,...,0.751750,93.000000,234.000000,743.625000,104.875000,45.000000,29.000000,NaN,266.500000,1138.375000
Zendon Hamilton,2004.125000,28.125000,18.250000,1.875000,201.125000,13.400000,0.493375,0.000000,1.132000,10.275000,...,0.679250,23.375000,36.000000,59.375000,4.125000,5.375000,3.500000,14.250000,27.500000,76.375000
Zoran Dragic,2015.000000,25.000000,10.666667,0.666667,50.000000,8.433333,0.412667,0.500333,0.211000,10.133333,...,0.589000,3.333333,2.000000,5.333333,3.333333,1.333333,0.000000,3.333333,4.000000,18.666667
